In [1]:
import pandas as pd

In [2]:
dm_ctt = pd.read_csv('/datasets/coupons big/DM/dm_content_generic_complete_from_2024.csv')
dm_cou = pd.read_csv('/datasets/coupons big/DM/coupons_dm.csv')

In [3]:
print(len(set(dm_cou['seen_timestamp'])))
print(len(set(dm_cou['seen_timestamp']).intersection(dm_ctt['seen_timestamp'])))

4497
4497


In [4]:
COLUMNS_REQUIRED_CTT = ['view_id', 'text', 'view_depth']
COLUMNS_REQUIRED_CPN = ['content_full', 'discount_text', 'activation_text', 'validity_text', 'product_text']

In [5]:
dm_ctt_new = dm_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)

In [6]:
dm_ctt_new['aggregation'] = dm_ctt['seen_timestamp']
# filling missing text fields with descriptions
dm_ctt_new.loc[dm_ctt_new['text'].isna(), 'text'] = dm_ctt.loc[dm_ctt_new['text'].isna(), 'description']
dm_ctt_new = dm_ctt_new[dm_ctt_new["aggregation"] != 0]

In [7]:
dm_cpn_new = dm_cou[COLUMNS_REQUIRED_CPN].copy(deep=True)

In [8]:
dm_cpn_new['aggregation'] = dm_cou['seen_timestamp']
dm_cpn_new = dm_cpn_new[dm_cpn_new["content_full"] != "['']"]
dm_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
dm_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
dm_ctt_new.reset_index(drop=True, inplace=True)
dm_cpn_new.reset_index(drop=True, inplace=True)

In [9]:
from src.bert_dataset_generation.generate_coupon_selection_ds import __samples_from_entry, drop_duplicates

In [10]:
samples_dm = __samples_from_entry(2, dm_ctt_new, dm_cpn_new, False)
samples_dm = drop_duplicates(samples_dm)

In [11]:
print(len([x for x in samples_dm if len(set(x[2])) > 1]) / len(samples_dm))

0.4493515597616544


In [12]:
dm_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/dm_content.csv')
dm_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/dm_coupons.csv')

In [13]:
edeka_ctt = pd.read_csv('/datasets/coupons big/Edeka/content_generic_edeka_2025_03_13.csv')
edeka_cpn = pd.read_csv('/datasets/coupons big/Edeka/coupons_edeka_2025-03-13T09_03_32.508347956+01_00.csv')

In [14]:
print(len(set(edeka_cpn['seen_timestamp'])))
print(len(set(edeka_cpn['seen_timestamp']).intersection(edeka_ctt['seen_timestamp'])))

181
1


In [15]:
print(len(set(edeka_cpn['id'])))
print(len(set(edeka_cpn['id']).intersection(edeka_ctt['id'])))

223
223


In [16]:
edeka_cpn_new = edeka_cpn[COLUMNS_REQUIRED_CPN].copy(deep=True)
edeka_cpn_new['aggregation'] = edeka_cpn['id']
edeka_ctt_new = edeka_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)
edeka_ctt_new['aggregation'] = edeka_ctt['id']
edeka_ctt_new.loc[edeka_ctt_new['text'].isna(), 'text'] = edeka_ctt.loc[edeka_ctt_new['text'].isna(), 'description']
edeka_ctt_new = edeka_ctt_new[edeka_ctt['seen_timestamp'] != 0]
edeka_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
edeka_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
edeka_ctt_new.reset_index(drop=True, inplace=True)
edeka_cpn_new.reset_index(drop=True, inplace=True)

contents_full = edeka_cpn_new['content_full'].tolist()
contents_full = [x[:-3] + ']' if x.endswith(', ]') else x for x in contents_full]
edeka_cpn_new['content_full'] = contents_full

In [17]:
samples_edeka = __samples_from_entry(1, edeka_ctt_new, edeka_cpn_new, False)
samples_edeka = drop_duplicates(samples_edeka)
print(len([x for x in samples_edeka if len(set(x[2])) > 1]) / len(samples_edeka))

0.38482384823848237


In [18]:
edeka_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/edeka_content.csv')
edeka_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/edeka_coupons.csv')

In [19]:
from src.llama_dataset_generation.ground_truth_parser import prepare_ground_truth_data_no_ai
from src.llama_dataset_generation.input_parser import create_training_df, prepare_input_data
from src.llama_dataset_generation.datasetter import MAP_FUNCTIONS, run_mapping

In [20]:
gt_edeka = prepare_ground_truth_data_no_ai(edeka_cpn_new)
_edeka_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/edeka_content.csv')
_llama_edeka = create_training_df(_edeka_input_data, gt_edeka)
llama_edeka = run_mapping(_llama_edeka, MAP_FUNCTIONS[3])

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [21]:
print(len([x for x in llama_edeka if x["Response"] != '[{}]']) / len(llama_edeka))

0.48509485094850946


In [22]:
print(len(samples_edeka))
print(len(llama_edeka))
print(len([x for x in llama_edeka if x["Response"] != '[{}]']))
print(len([x for x in samples_edeka if len(set(x[2])) > 1]))

369
369
179
142


The missing examples with coupon in bert samples are a result of missing row with price in content frame (see aggregation=1861390104368381994). <br/>
We will assume that there is regularity in this anomaly and try to match the coupon with price removed.

In [23]:
aggregation_values = set([x["Aggregation"] for x in llama_edeka if x["Response"] != '[{}]']).difference(set(str(x[0]) for x in samples_edeka if len(set(x[2])) > 1))
_cpn_new = []
_discounts_new = []
for i, row in edeka_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        try:
            cf = "[Main logo, " + cf[cf.index("APP Preis"):]
            _cpn_new.append(cf)
            _discounts_new.append(float('nan'))
        except:
            _cpn_new.append(row['content_full'])
            _discounts_new.append(row['discount_text'])
    else:
        _cpn_new.append(row['content_full'])
        _discounts_new.append(row['discount_text'])
edeka_cpn_new['content_full'] = _cpn_new
edeka_cpn_new['discount_text'] = _discounts_new
edeka_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/edeka_coupons.csv')
samples_edeka = __samples_from_entry(1, edeka_ctt_new, edeka_cpn_new, False)
samples_edeka = drop_duplicates(samples_edeka)
print(len([x for x in samples_edeka if len(set(x[2])) > 1]) / len(samples_edeka))

0.4128686327077748


In [24]:
gt_dm = prepare_ground_truth_data_no_ai(dm_cpn_new)
_dm_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/dm_content.csv')
_llama_dm = create_training_df(_dm_input_data, gt_dm)
llama_dm = run_mapping(_llama_dm, MAP_FUNCTIONS[3])
print(print(len([x for x in llama_dm if x["Response"] != '[{}]']) / len(llama_dm)))

Map:   0%|          | 0/2849 [00:00<?, ? examples/s]

0.5110565110565111
None


In [25]:
aggregation_values = set([x["Aggregation"] for x in llama_dm if x["Response"] != '[{}]']).difference(set(str(x[0]) for x in samples_dm if len(set(x[2])) > 1))

In [26]:
for i, row in dm_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        if "''" not in cf:
            print('!'*20)
            print(agg)
            print(cf)
            print('!'*20)
        else:
            print(agg)
            print(cf)

1731309241898
['','NEU']
1731309238586
['','NEU','']
1731309241898
['Kneipp Badezusätze. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Kneipp Badezusätze','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','','20%','','Gültig bis 03.12.2024','Coupon aktivieren']
1731309245530
['Borotalco. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Borotalco','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','','20%','','Gültig bis 03.12.2024','Coupon aktivieren']
1731309245790
['','NEU','']
1731309245790
['Borotalco. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Borotalco','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','','20%','','Gültig bis 03.12.2024','Coupon aktivieren']
1731309248542
['','NEU','']
1731309250108
['','NEU','']
1731309253009
['HEJ Protein-Riegel. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 29.11.2024','HEJ

Almost every coupon not found by coupon selection ds generation has empty string in content full. We will try to remove those

In [27]:
_cpn_new = []
for i, row in dm_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        try:
            print(cf)
            cf = eval(cf)
            cf = [x for x in cf if x]
            cf = str(cf).replace("', '", "','")
            print(cf)
            print("="*20)
            _cpn_new.append(cf)
        except:
            _cpn_new.append(row['content_full'])
    else:
        _cpn_new.append(row['content_full'])
dm_cpn_new['content_full'] = _cpn_new
dm_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/dm_coupons.csv')
samples_dm = __samples_from_entry(2, dm_ctt_new, dm_cpn_new, False)
samples_dm = drop_duplicates(samples_dm)
print(len([x for x in samples_dm if len(set(x[2])) > 1]) / len(samples_dm))

['','NEU']
['NEU']
['','NEU','']
['NEU']
['Kneipp Badezusätze. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Kneipp Badezusätze','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','','20%','','Gültig bis 03.12.2024','Coupon aktivieren']
['Kneipp Badezusätze. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Kneipp Badezusätze','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','20%','Gültig bis 03.12.2024','Coupon aktivieren']
['Borotalco. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Borotalco','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','','20%','','Gültig bis 03.12.2024','Coupon aktivieren']
['Borotalco. Gilt für beliebig viele Artikel innerhalb eines Einkaufs 20%. Gültig bis 03.12.2024','Borotalco','Gilt für beliebig viele Artikel innerhalb eines Einkaufs','20%','Gültig bis 03.12.2024','Coupon aktivieren']
['','NEU','']
['NEU']
['Borotal

i am fine with the results

In [28]:
lidl_ctt = pd.read_csv('/datasets/coupons big/Lidl/content_generic_lidl_2025_03_13.csv')
lidl_cpn = pd.read_csv('/datasets/coupons big/Lidl/coupons_lidl_2025-03-13T09_40_11.213120937+01_00.csv')

In [29]:
print(len(set(lidl_cpn["seen_timestamp"])))
print(len(set(lidl_ctt["seen_timestamp"]).intersection(lidl_cpn["seen_timestamp"])))
print(len(set(lidl_cpn["time"])))
print(len(set(lidl_ctt["time"]).intersection(lidl_cpn["time"])))
print(len(set(lidl_cpn["id"])))
print(len(set(lidl_ctt["id"]).intersection(lidl_cpn["id"])))

2423
10
3107
0
3175
3175


In [30]:
lidl_cpn_new = lidl_cpn[COLUMNS_REQUIRED_CPN].copy(deep=True)
lidl_cpn_new['aggregation'] = lidl_cpn['id']
lidl_ctt_new = lidl_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)
lidl_ctt_new['aggregation'] = lidl_ctt['id']
lidl_ctt_new.loc[lidl_ctt_new['text'].isna(), 'text'] = lidl_ctt.loc[lidl_ctt_new['text'].isna(), 'description']
lidl_ctt_new = lidl_ctt_new[lidl_ctt['seen_timestamp'] != 0]
lidl_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
lidl_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
lidl_ctt_new.reset_index(drop=True, inplace=True)
lidl_cpn_new.reset_index(drop=True, inplace=True)

samples_lidl = __samples_from_entry(1, lidl_ctt_new, lidl_cpn_new, False)
samples_lidl = drop_duplicates(samples_lidl)
print(len([x for x in samples_lidl if len(set(x[2])) > 1]) / len(samples_lidl))

lidl_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/lidl_content.csv')
lidl_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/lidl_coupons.csv')

gt_lidl = prepare_ground_truth_data_no_ai(lidl_cpn_new)
_lidl_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/lidl_content.csv')
_llama_lidl = create_training_df(_lidl_input_data, gt_lidl)
llama_lidl = run_mapping(_llama_lidl, MAP_FUNCTIONS[3])
print(print(len([x for x in llama_lidl if x["Response"] != '[{}]']) / len(llama_lidl)))

0.7223340040241448


Map:   0%|          | 0/3979 [00:00<?, ? examples/s]

0.7253078662980649
None


In [31]:
aggregation_values = set([x["Aggregation"] for x in llama_lidl if x["Response"] != '[{}]']).difference(set(str(x[0]) for x in samples_lidl if len(set(x[2])) > 1))
for i, row in lidl_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        print(agg)
        print(cf)

14956303685
[ONLINE COUPON, 5.95€ sparen, Ab 79€ Mindestbestellwert, 5.95€ Versandkosten sparen*, Noch 18 Tage gültig, AKTIVIEREN]
14956303685
[ONLINE COUPON, -30%, Deal des Tages, Hisense Smart TV 75 Zoll 4K Mini LED ULED »U8HQ«\n, Nur noch heute gültig, AKTIVIEREN]
14956303685
[ONLINE COUPON, -47%, Deal des Tages, HUDORA Trampolin »Fantastic 300V«, 300 cm Durchmesser, 100 kg Belastbarkeit, Nur noch heute gültig, AKTIVIEREN]
14956303686
[ONLINE COUPON, 5.95€ sparen, Ab 79€ Mindestbestellwert, 5.95€ Versandkosten sparen*, Noch 18 Tage gültig, AKTIV]
14956303686
[ONLINE COUPON, -47%, Deal des Tages, HUDORA Trampolin »Fantastic 300V«, 300 cm Durchmesser, 100 kg Belastbarkeit, Nur noch heute gültig, AKTIVIEREN]
14956303686
[ONLINE COUPON, -30%, Deal des Tages, Hisense Smart TV 75 Zoll 4K Mini LED ULED »U8HQ«\n, Nur noch heute gültig, AKTIVIEREN]
14956303691
[ONLINE COUPON, 5.95€ sparen, Ab 79€ Mindestbestellwert, 5.95€ Versandkosten sparen*, Noch 18 Tage gültig, AKTIV]
14956303696
[ONLINE

In [32]:
lidl_ctt_new[lidl_ctt_new['aggregation'] == 15370178330]

,view_id,text,view_depth,aggregation
358000,com.lidl.eci.lidlplus:id/action_bar_root,NaN,3,15370178330
358001,android:id/content,NaN,4,15370178330
358002,com.lidl.eci.lidlplus:id/main_coordinator_layout,NaN,5,15370178330
358003,com.lidl.eci.lidlplus:id/placeSnackBar,NaN,6,15370178330
358004,com.lidl.eci.lidlplus:id/main_fragment_container,NaN,8,15370178330
...,...,...,...,...
358080,com.lidl.eci.lidlplus:id/discount_description,Deal des Tages,14,15370178330
358081,com.lidl.eci.lidlplus:id/title,LOOKS by Wolfgang Joop Badmöbel-Set (3-teilig),14,15370178330
358082,com.lidl.eci.lidlplus:id/expiration,Nur noch heute gültig,14,15370178330
358083,com.lidl.eci.lidlplus:id/activation_button,AKTIVIEREN,14,15370178330


We see that there are missing "Hier zu lidl.de" texts in content_full. Lets add them and check if we get more matches

In [33]:
_cpn_new = []
for i, row in lidl_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        try:
            cf = cf.replace("[ONLINE COUPON, ", "[ONLINE COUPON, Hier zu lidl.de, ", 1)
            _cpn_new.append(cf)
        except:
            _cpn_new.append(row['content_full'])
    else:
        _cpn_new.append(row['content_full'])
lidl_cpn_new['content_full'] = _cpn_new
lidl_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/lidl_coupons.csv')
samples_lidl = __samples_from_entry(1, lidl_ctt_new, lidl_cpn_new, False)
samples_lidl = drop_duplicates(samples_lidl)
print(len([x for x in samples_lidl if len(set(x[2])) > 1]) / len(samples_lidl))

gt_lidl = prepare_ground_truth_data_no_ai(lidl_cpn_new)
_lidl_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/lidl_content.csv')
_llama_lidl = create_training_df(_lidl_input_data, gt_lidl)
llama_lidl = run_mapping(_llama_lidl, MAP_FUNCTIONS[3])
print(print(len([x for x in llama_lidl if x["Response"] != '[{}]']) / len(llama_lidl)))

0.7258551307847082


Map:   0%|          | 0/3979 [00:00<?, ? examples/s]

0.7253078662980649
None


In [54]:
print(len(set(x['text'] for x in llama_lidl)))
print(len(set(x['Aggregation'] for x in llama_lidl)))
print(len(llama_lidl))
print(len([x for x in llama_lidl if x["Response"] != '[{}]']))

print(len(samples_lidl))
print(len([x for x in samples_lidl if len(set(x[2])) > 1]))
print(len(set(str(x[0]) for x in samples_lidl)))

3979
3979
3979
2886
3976
2886
3976


In [35]:
penny_ctt = pd.read_csv('/datasets/coupons big/Penny/content_generic_penny_2025_03_13.csv')
penny_cpn = pd.read_csv('/datasets/coupons big/Penny/coupons_penny_cleared.csv')

In [36]:
print(len(set(penny_cpn["seen_timestamp"])))
print(len(set(penny_ctt["seen_timestamp"]).intersection(penny_cpn["seen_timestamp"])))
print(len(set(penny_cpn["time"])))
print(len(set(penny_ctt["time"]).intersection(penny_cpn["time"])))
print(len(set(penny_cpn["id"])))
print(len(set(penny_ctt["id"]).intersection(penny_cpn["id"])))

41
25
43
0
44
44


In [37]:
penny_cpn_new = penny_cpn[COLUMNS_REQUIRED_CPN].copy(deep=True)
penny_cpn_new['aggregation'] = penny_cpn['id']
penny_ctt_new = penny_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)
penny_ctt_new['aggregation'] = penny_ctt['id']
#penny_ctt_new.loc[penny_ctt_new['text'].isna(), 'text'] = penny_ctt.loc[penny_ctt_new['text'].isna(), 'description']
penny_ctt_new = penny_ctt_new[penny_ctt['seen_timestamp'] != 0]
penny_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
penny_ctt_new.reset_index(drop=True, inplace=True)
penny_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
penny_cpn_new.reset_index(drop=True, inplace=True)

samples_penny = __samples_from_entry(1, penny_ctt_new, penny_cpn_new, False)
samples_penny = drop_duplicates(samples_penny)
print(len([x for x in samples_penny if len(set(x[2])) > 1]) / len(samples_penny))

penny_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/penny_content.csv')
penny_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/penny_coupons.csv')

gt_penny = prepare_ground_truth_data_no_ai(penny_cpn_new)
_penny_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/penny_content.csv')
_llama_penny = create_training_df(_penny_input_data, gt_penny)
llama_penny = run_mapping(_llama_penny, MAP_FUNCTIONS[3])
print(print(len([x for x in llama_penny if x["Response"] != '[{}]']) / len(llama_penny)))

0.19791666666666666


Map:   0%|          | 0/96 [00:00<?, ? examples/s]

0.3541666666666667
None


In [38]:
aggregation_values = set([x["Aggregation"] for x in llama_penny if x["Response"] != '[{}]']).difference(set(str(x[0]) for x in samples_penny if len(set(x[2])) > 1))
missing = []
for i, row in penny_cpn_new.iterrows():
    if str(row["aggregation"]) in aggregation_values:
        agg = row["aggregation"]
        cf = row["content_full"]
        print(agg, cf)
        missing.append((i, row))

18144690552 [PENNY Butter- oder Vollkorntoast, 22%, 0.99, 0.77, Gültig bis Sa, 10.8.]
18144740275 [EHRMANN Almighurt Joghurt, 63%, 0.79, 0.28, Gültig bis Sa, 10.8.]
18144740275 [LEERDAMMER Käsescheiben, 42%, 2.59, 1.49, Gültig bis Sa, 10.8.]
18144740275 [MARKTLIEBE Rote Paprika, 33%, 2.99, 1.99, Gültig bis Sa, 10.8.]
18171633170 [BOSCH 2-in-1-Akku-Staubsauger BCHF216B, 56%, 204.99, 89.99, Gültig bis Sa, 10.8.]
18171633170 [GRILLPARTY Stadionbratwurst, 33%, 4.49, 2.99, Gültig bis Sa, 10.8.]
18171633170 [ORIGINAL WAGNER Piccolinis, 49%, 3.49, 1.75, Gültig bis Sa, 10.8.]
18171877585 [NATURGUT Bio Feta, 23%, 2.09, 1.59, Gültig bis Sa, 10.8.]
18171877585 [ORIGINAL WAGNER Piccolinis, 49%, 3.49, 1.75, Gültig bis Sa, 10.8.]
18172158098 [BARILLA Pesto, 51%, 3.79, 1.85, Gültig bis Sa, 10.8.]
18172158098 [EHRMANN Almighurt Joghurt, 63%, 0.79, 0.28, Gültig bis Sa, 10.8.]
18172158098 [PENNY READY Frikadellenbällchen, 13%, 2.99, 2.59, Gültig bis Sa, 10.8.]
18172158103 [BARILLA Pesto, 51%, 3.79, 1.85

looks like in the part of the data from penny `content_full` has different meaning than we assumed. I need to fix that

In [39]:
for i, row in missing:
    agg = row["aggregation"]
    print(agg)
    texts = penny_ctt_new[penny_ctt_new["aggregation"] == agg]["text"].tolist()
    for ix, txt in enumerate(texts):
        if pd.isna(txt):
            continue
        if row["content_full"][1:].startswith(txt.strip()):
            if texts[ix - 3] == 'GERSTACKER Federmost':
                ncf = texts[ix-6:ix+1]
            else:
                ncf = texts[ix-3:ix+2]
            row["content_full"] = '[' + ', '.join(ncf) + ']'
            break
    else:
        print('not found')
        print(row)
        print(texts)
    penny_cpn_new.loc[i] = row

18144690552
18144740275
18144740275
18144740275
18171633170
18171633170
18171633170
18171877585
18171877585
18172158098
18172158098
18172158098
18172158103
18172158103
18172158132
18172158132
18172489271
18172489271
18772356980
18772356980
18773097214
18773097214
18796067278
18796067278
18798286962
18798286962
18801243572
18801243572
1846149031203307530
1846149031203307530
1846575669952380950
1846575669952380950


In [40]:
penny_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/penny_coupons.csv')

samples_penny = __samples_from_entry(1, penny_ctt_new, penny_cpn_new, False)
samples_penny = drop_duplicates(samples_penny)
print(len([x for x in samples_penny if len(set(x[2])) > 1]) / len(samples_penny))

gt_penny = prepare_ground_truth_data_no_ai(penny_cpn_new)
_penny_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/penny_content.csv')
_llama_penny = create_training_df(_penny_input_data, gt_penny)
llama_penny = run_mapping(_llama_penny, MAP_FUNCTIONS[3])
print(print(len([x for x in llama_penny if x["Response"] != '[{}]']) / len(llama_penny)))

0.3434343434343434


Map:   0%|          | 0/96 [00:00<?, ? examples/s]

0.3541666666666667
None


In [41]:
print(len(llama_penny))
print(len(samples_penny))
print(set([str(x[0]) for x in samples_penny]).difference(set([x["Aggregation"] for x in llama_penny])))

96
99
{'1846575669952380951', '18772356981', '18801243564'}


In [42]:
rewe_ctt = pd.read_csv('/datasets/coupons big/Rewe/content_generic_rewe_2025_03_13.csv')
rewe_cpn = pd.read_csv('/datasets/coupons big/Rewe/coupons_rewe_2025-03-13T08_58_21.509638957+01_00.csv')

In [43]:
print(len(set(rewe_cpn["seen_timestamp"])))
print(len(set(rewe_ctt["seen_timestamp"]).intersection(rewe_cpn["seen_timestamp"])))
print(len(set(rewe_cpn["time"])))
print(len(set(rewe_ctt["time"]).intersection(rewe_cpn["time"])))
print(len(set(rewe_cpn["id"])))
print(len(set(rewe_ctt["id"]).intersection(rewe_cpn["id"])))

3643
3643
3643
0
3022
3022


In [44]:
rewe_cpn_new = rewe_cpn[COLUMNS_REQUIRED_CPN].copy(deep=True)
rewe_cpn_new['aggregation'] = rewe_cpn['seen_timestamp']
rewe_ctt_new = rewe_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)
rewe_ctt_new['aggregation'] = rewe_ctt['seen_timestamp']
rewe_ctt_new.loc[rewe_ctt_new['text'].isna(), 'text'] = rewe_ctt.loc[rewe_ctt_new['text'].isna(), 'description']
rewe_ctt_new = rewe_ctt_new[rewe_ctt['seen_timestamp'] != 0]
rewe_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
rewe_ctt_new.reset_index(drop=True, inplace=True)
rewe_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
rewe_cpn_new.reset_index(drop=True, inplace=True)

samples_rewe = __samples_from_entry(1, rewe_ctt_new, rewe_cpn_new, False)
samples_rewe = drop_duplicates(samples_rewe)
print(len(samples_rewe), len([x for x in samples_rewe if len(set(x[2])) > 1]) / len(samples_rewe))

rewe_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rewe_content.csv')
rewe_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rewe_coupons.csv')

gt_rewe = prepare_ground_truth_data_no_ai(rewe_cpn_new)
_rewe_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/rewe_content.csv')
_llama_rewe = create_training_df(_rewe_input_data, gt_rewe)
llama_rewe = run_mapping(_llama_rewe, MAP_FUNCTIONS[3])
print(len(llama_rewe), len([x for x in llama_rewe if x["Response"] != '[{}]']) / len(llama_rewe))

2169 0.6371599815583218


Map:   0%|          | 0/2160 [00:00<?, ? examples/s]

2160 0.6351851851851852


In [45]:
s = set([str(x[0]) for x in samples_rewe]).difference(set(x['Aggregation'] for x in llama_rewe))
print(s)
print(len(s))

{'1733147785839', '1716385103551', '1728580416667', '1728580412956', '1717834489195', '1712853139611', '1712071327772', '1711742040758', '1712853139506', '1725030946112', '1706173326121', '1712853137614', '1729069331678'}
13


In [46]:
texts = {}
samples_to_erase = []
for x in samples_rewe:
    joint = ' '.join(x[1])
    if joint in texts:
        if x[2].count(1) < texts[joint][0]:
            samples_to_erase.append(x[0])
        elif x[2].count(1) == texts[joint][0]:
            texts[joint][1].append(x[0])
        else:
            samples_to_erase.extend(texts[joint][1])
            texts[joint] = (x[2].count(1), [x[0]])
    else:
        texts[joint] = (x[2].count(1), [x[0]])

In [47]:
samples_to_erase

[1706173326121,
 1711742040758,
 1712071327772,
 1712853137614,
 1712853139359,
 1712853139506,
 1716385103551,
 1717834489195,
 1725030946112,
 1728580412956,
 1728580416667,
 1729069331678,
 1733147785839]

In [48]:
print(len(rewe_cpn_new))
rewe_cpn_new = rewe_cpn_new[~rewe_cpn_new['aggregation'].isin(samples_to_erase)]
rewe_cpn_new.reset_index(drop=True, inplace=True)
len(rewe_cpn_new)

30759


30578

In [49]:
print(len(rewe_ctt_new))
rewe_ctt_new = rewe_ctt_new[~rewe_ctt_new['aggregation'].isin(samples_to_erase)]
rewe_ctt_new.reset_index(drop=True, inplace=True)
len(rewe_ctt_new)

433323


430947

In [50]:
samples_rewe = __samples_from_entry(1, rewe_ctt_new, rewe_cpn_new, False)
samples_rewe = drop_duplicates(samples_rewe)
print(len(samples_rewe), len([x for x in samples_rewe if len(set(x[2])) > 1]) / len(samples_rewe))

rewe_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rewe_content.csv')
rewe_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rewe_coupons.csv')

gt_rewe = prepare_ground_truth_data_no_ai(rewe_cpn_new)
_rewe_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/rewe_content.csv')
_llama_rewe = create_training_df(_rewe_input_data, gt_rewe)
llama_rewe = run_mapping(_llama_rewe, MAP_FUNCTIONS[3])
print(len(llama_rewe), len([x for x in llama_rewe if x["Response"] != '[{}]']) / len(llama_rewe))

2156 0.6354359925788498


Map:   0%|          | 0/2160 [00:00<?, ? examples/s]

2160 0.6351851851851852


In [51]:
rossmann_ctt = pd.read_csv('/datasets/coupons big/Rossmann/content_generic_rossmann_2025_03_13.csv')
rossmann_cpn = pd.read_csv('/datasets/coupons big/Rossmann/coupons_rossmann_2025-03-13T09_25_10.66644692+01_00.csv')

In [52]:
print(len(set(rossmann_cpn["seen_timestamp"])))
print(len(set(rossmann_ctt["seen_timestamp"]).intersection(rossmann_cpn["seen_timestamp"])))
print(len(set(rossmann_cpn["time"])))
print(len(set(rossmann_ctt["time"]).intersection(rossmann_cpn["time"])))
print(len(set(rossmann_cpn["id"])))
print(len(set(rossmann_ctt["id"]).intersection(rossmann_cpn["id"])))

1026
2
1670
0
1679
1679


In [53]:
rossmann_cpn_new = rossmann_cpn[COLUMNS_REQUIRED_CPN].copy(deep=True)
rossmann_cpn_new['aggregation'] = rossmann_cpn['id']
rossmann_ctt_new = rossmann_ctt[COLUMNS_REQUIRED_CTT].copy(deep=True)
rossmann_ctt_new['aggregation'] = rossmann_ctt['id']
rossmann_ctt_new.loc[rossmann_ctt_new['text'].isna(), 'text'] = rossmann_ctt.loc[rossmann_ctt_new['text'].isna(), 'description']
rossmann_ctt_new = rossmann_ctt_new[rossmann_ctt['seen_timestamp'] != 0]
rossmann_ctt_new.sort_values(by=['aggregation'], inplace=True, kind='stable')
rossmann_ctt_new.reset_index(drop=True, inplace=True)
rossmann_cpn_new.drop_duplicates(subset=COLUMNS_REQUIRED_CPN + ['aggregation'], inplace=True)
rossmann_cpn_new.reset_index(drop=True, inplace=True)

samples_rossmann = __samples_from_entry(1, rossmann_ctt_new, rossmann_cpn_new, False)
samples_rossmann = drop_duplicates(samples_rossmann)
print(len(samples_rossmann), len([x for x in samples_rossmann if len(set(x[2])) > 1]) / len(samples_rossmann))

rossmann_ctt_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rossmann_content.csv')
rossmann_cpn_new.to_csv('/home/szymon/murmuras/ZPP_murmuras/local/rossmann_coupons.csv')

gt_rossmann = prepare_ground_truth_data_no_ai(rossmann_cpn_new)
_rossmann_input_data = prepare_input_data('/home/szymon/murmuras/ZPP_murmuras/local/rossmann_content.csv')
_llama_rossmann = create_training_df(_rossmann_input_data, gt_rossmann)
llama_rossmann = run_mapping(_llama_rossmann, MAP_FUNCTIONS[3])
print(len(llama_rossmann), len([x for x in llama_rossmann if x["Response"] != '[{}]']) / len(llama_rossmann))

1846 0.7351029252437703


Map:   0%|          | 0/1846 [00:00<?, ? examples/s]

1846 0.7351029252437703
